In [ ]:
# Basic setup and sanity checking
%run ./00_setup.ipynb

## Create a data set for Training Data

Create a data set for Visual Recognition training. These are input files or folders that typically exist in your local file system or in S3.

In [ ]:
from cortex.dataset import Dataset
from cortex.connection import ConnectionClient
import os
from os.path import join
cortex = Cortex.client()
builder = cortex.builder()

positive_example_1 = ''

for filename in os.listdir('./data/images'):
    fullpath = join('./data/images',filename)
    datasetbuilder = builder.dataset("lkrishna/" + filename)
    datasetbuilder.title(filename)
    with open(fullpath,'rb') as f:
        datasetbuilder.from_file(f,format='zip').build()

## Action Authoring and Deployment
A Cortex Action can be authored and deployed from within a notebook for rapid prototyping and testing.  The Cortex SDK for Python includes an iPython Magic called `%%cortex_action` that takes a notebook cell and deploys it as a Cortex Action.

In [ ]:
%%cortex_action --name c12e/image-train --function train --daemon --requirements watson_developer_cloud

from cortex import Message
from watson_developer_cloud import VisualRecognitionV3
import cortex
import io

def get_dataset_refs(params):
    return {
        x["datasetName"]: x["refName"] for x in params["datasetBindings"]
    }

def read_dataset(args, jwt, dsname):

    URL = 'https://api.cortex.insights.ai'
    dc = cortex.dataset.DatasetsClient(URL, "3", jwt)
    data = dc.get_stream(dsname).read()
    return data
    
def train(params):


    msg = Message(params)
    
    #datasetReferences = get_dataset_refs(params)
    
    #parsedArgs = {
    #    HDMI_FILE_REF: datasetReferences.get(HDMI_FILE_REF),
    #    "api_endpoint": params.get("apiEndpoint")
    #   }
    
    parsedArgs = None
    jwt = params.get("token")
    
    apiKey = msg.payload.get('apiKey')
    classifier_name = msg.payload.get('classifier_name')
    datasets = msg.payload.get('datasets')
    classes = msg.payload.get('classes')
    
    if classifier_name is None or classifier_name == "":
        return {"payload": "Classifier Name Not Provided"}
    
    # Store all files
    image_files = []
    count = 0
    for dataset in datasets:
        file_name = '/tmp/file_'+ str(count) + '.zip'
        file = open(file_name,'wb')
        file.write(read_dataset(parsedArgs, jwt, dataset))
        image_files.append(file_name)
        count=count+1
    
    # Get Watson VR
    visual_recognition = VisualRecognitionV3('2018-03-19', iam_apikey=apiKey)
    
    positive_examples = {}
    
    count = 0
    for class_name in classes:
        file_name = class_name + "_positive_examples"
        file = open(image_files[count],'rb')
        positive_examples[file_name] = file
        count=count+1

    model = visual_recognition.create_classifier(
        classifier_name,
        **positive_examples).get_result()
    response = json.dumps(model, indent=2)
    
    if response is not None:
        return {"payload": response}
    else:
        return {"payload": "Unable to get response from Watson Visual Recognition"}        


## Local Testing
Once the Action is built, we can run both a local and remote test.  The local test in our example can be done by simply calling the `detect` function we defined above.

In [ ]:
import sys
import argparse
from pprint import pprint


def test_train(args):
    classifier_name = 'Connectors'
    apiKey = 'BVaceevn9oDep6ow12MASoLubdwkLYkDYEKdjLdBKFRi'
    m = Message.with_payload({'apiKey':apiKey,'classifier_name':classifier_name,'classes':['thunderbolt_male','hdmi_male','usb_male','vga_male'],'datasets':['lkrishna/hdmi_male.zip','lkrishna/thunderbolt_male.zip','lkrishna/vga_male.zip','lkrishna/usb_male.zip']})
    result = train(m.to_params())
    pprint(result)

test_train(None)

## Remote Testing
Using the Cortex client, we can execute a remote test of our Action to make sure it deployed as expected.

In [ ]:
client = Cortex.client()
action = client.action('c12e/image-train')
action

In [ ]:
classifier_name = 'Connectors'
apiKey = 'BVaceevn9oDep6ow12MASoLubdwkLYkDYEKdjLdBKFRi'
params = {'payload': {'apiKey':apiKey,'classifier_name':classifier_name,'datasets':['lkrishna/thunderbolt_male.zip','lkrishna/hdmi_male.zip','lkrishna/usb_male.zip','lkrishna/vga_male.zip'],'classes':['thunderbolt_male','hdmi_male','usb_male','vga_male']}}
rs = action.invoke(client.message(params.get('payload')))
result = rs.payload
print(result)

## Get Status of Training